# Uncertainty Losses Iteration #1

In [1]:
import os
import sys

sys.path.append(os.path.abspath('..'))

In [2]:
import random

import numpy as np
from copy import copy
import skml
from skml.problem_transformation import ProbabilisticClassifierChain
from skml.datasets import sample_down_label_space
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.base import clone
from sklearn.externals import joblib
import arff

from lib.experimental_framework import load_from_arff
random.seed(2018)

In [3]:
X, y = load_from_arff('../data/enron/enron.arff', labelcount=53, endian='little')
y = sample_down_label_space(y, k=10)

X = X.todense()
y = y.todense()

In [4]:
try:
    results = joblib.load('../data/predictions.pkl')
    idxs = joblib.load('../data/idxs.pkl')
except:
    print("re-training")
    results = []
    idxs = []

In [5]:
if len(results) == 0:
    clf = ProbabilisticClassifierChain(LogisticRegression())
    kf = KFold()

    results = []

    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        idxs.append((train_index, test_index))
        pcc = clone(clf)

        pcc.fit(X_train, y_train)
        y_pred = pcc.predict(X_test)
        y_pred_pp = pcc.predict_proba(X_test)
        results.append((y_pred, y_pred_pp))

        print("----------")

        print("hamming loss: ")
        print(hamming_loss(y_test, y_pred))

        print("accuracy:")
        print(accuracy_score(y_test, y_pred))

        print("f1 score:")
        print("micro")
        print(f1_score(y_test, y_pred, average='micro'))
        print("macro")
        print(f1_score(y_test, y_pred, average='macro'))

        print("precision:")
        print("micro")
        print(precision_score(y_test, y_pred, average='micro'))
        print("macro")
        print(precision_score(y_test, y_pred, average='macro'))

        print("recall:")
        print("micro")
        print(recall_score(y_test, y_pred, average='micro'))
        print("macro")
        print(recall_score(y_test, y_pred, average='macro'))y

    joblib.dump(results, '../data/predictions.pkl')
    joblib.dump(idxs, '../data/idxs.pkl')

In [92]:
def uncertain_hamming_loss(y, y_pred, omega=1.0):
    N, L = y_pred.shape
    cumsum = 0
    
    # place '?' if not already done
    np.place(y_pred, 
             mask=np.logical_and(y_pred > 1/3, y_pred < 2/3),
             vals=np.nan)
    u = np.isnan(y_pred).sum()
    hl = (y_pred.round() != y.astype(float)).sum()
    print(hl / (N * L))
    print("avg u:", u / N)
    return (hl + (u * omega)) / (N * L)

In [103]:
for split, res in zip(idxs, results):
    y_pred, y_pred_pp = res
    test_index = split[1]
    y_test = y[test_index].A
    y_pred_pp = y_pred_pp.reshape(y_pred_pp.shape[0], y_pred_pp.shape[2])

    print("-> hamming loss:")
    print(hamming_loss(y_test, y_pred_pp >= .5))
    print("-> uncertain hamming loss:")
    print(uncertain_hamming_loss(y_test, y_pred_pp))
    print("---")

    """
    # should be .9
    print(uncertain_hamming_loss(y_gt, y_test, .3))
    y_test_2 = copy(y_test)
    y_test_2[0][1] = np.nan
    # print(y_test_2)
    print(uncertain_hamming_loss(y_gt, y_test_2, .3))
    """

-> hamming loss:
0.7767605633802817
-> uncertain hamming loss:
0.8186619718309859
avg u: 0.8767605633802817
0.9063380281690141
---
-> hamming loss:
0.735978835978836
-> uncertain hamming loss:
0.7881834215167548
avg u: 0.9171075837742504
0.8798941798941798
---
-> hamming loss:
0.7313932980599648
-> uncertain hamming loss:
0.7790123456790123
avg u: 0.9118165784832452
0.8701940035273369
---


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater_equal
  
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys
/usr/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in less
  import sys


In [82]:
test = ['?', 1., 0., '?']
gt = [0., 1., 0., 1.]
y_test = results[0][1][0]

In [8]:
# this places the '?' as np.nan
np.place(y_test, mask=np.logical_and(y_test > 1/3, y_test < 2/3), vals=np.nan)

In [9]:
y_test

array([[0.96297518, 0.89641911, 0.91614432,        nan, 0.93988057,
        0.95834067, 0.88272036, 0.93995837, 0.85116763, 0.81247914]])

In [15]:
y_gt = y[0].A[0]
y_gt

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 1], dtype=int64)

In [16]:
y_test.round()

array([[ 1.,  1.,  1., nan,  1.,  1.,  1.,  1.,  1.,  1.]])

In [17]:
y_gt.astype(float)

array([0., 0., 0., 1., 0., 0., 0., 0., 1., 1.])

In [18]:
hamming_loss([1] * 10, y_gt.reshape(1, -1)[0])

0.7

In [19]:
1/10*(y_test.round() != y_gt.astype(float)).sum()

0.8

In [36]:
len(results)

3

In [26]:
res[0][0], res[1][0]

(array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0]),
 array([[0.96297518, 0.89641911, 0.91614432,        nan, 0.93988057,
         0.95834067, 0.88272036, 0.93995837, 0.85116763, 0.81247914]]))